In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense



In [2]:
#  dataset loading
data = pd.read_csv(r"C:\Users\himan\Desktop\7th GAN\updataDataSets.csv")
data

,SL NO,From Date,PM2.5,PM10,NO,NO2,Nox,NH3,SO2,CO,Ozone,Benzene,Eth-Benzene,MP-Xylene,WS,WD,SR,BP,AT,RF
0,1,18-02-2019 00:00,58.04,81.06,1.81,6.40,9.66,27.18,13.77,0.49,15.73,0.92,NaN,0.33,1.86,229.44,0.22,1002.19,20.87,0.00
1,2,18-02-2019 01:00,60.00,82.39,1.91,6.08,9.70,26.46,13.64,0.44,18.36,0.84,NaN,0.29,1.54,217.20,1.46,956.00,18.92,0.24
2,3,18-02-2019 02:00,55.50,73.42,2.22,6.71,10.86,27.06,13.64,0.44,34.54,0.77,NaN,0.23,0.66,197.21,NaN,1001.60,18.33,0.00
3,4,18-02-2019 03:00,66.56,96.04,2.79,7.00,12.10,26.34,13.72,0.45,31.85,0.82,NaN,0.21,0.61,206.44,NaN,1001.41,17.63,0.00
4,5,18-02-2019 04:00,67.08,97.21,3.29,9.45,15.53,25.16,13.80,0.47,24.07,0.74,NaN,0.22,0.54,230.53,NaN,980.67,16.61,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33062,33063,30-12-2022 20:00,295.72,371.83,5.54,2.62,5.41,16.13,13.91,2.64,22.82,2.96,3.16,3.37,0.31,168.12,NaN,1009.22,15.96,0.00
33063,33064,30-12-2022 21:00,362.60,448.50,5.61,2.62,5.46,17.45,13.88,2.79,5.66,3.12,3.34,3.56,0.69,132.27,11.17,967.18,14.93,0.48
33064,33065,30-12-2022 22:00,277.86,353.78,5.52,2.46,5.29,17.92,13.74,3.25,22.64,3.64,3.89,4.15,0.36,173.64,NaN,1009.11,15.22,0.00
33065,33066,30-12-2022 23:00,380.00,450.48,5.46,2.47,5.32,18.01,14.12,3.28,7.07,3.68,3.94,4.21,0.82,163.19,9.57,912.74,13.45,1.09


In [3]:
# Dropping unnecessary columns like 'SL NO' and 'From Date'
data = data.drop(columns=['SL NO', 'From Date'])

In [4]:
# Replaceing NaN values with 0
data.fillna(0, inplace=True)

In [5]:
#  dataset has columns for features and a target variable

feature_columns = ['PM10', 'NO', 'NO2', 'Nox', 'NH3', 'SO2', 'CO', 'Ozone', 'Benzene', 'Eth-Benzene', 'MP-Xylene', 'WS', 'WD', 'SR', 'BP', 'AT', 'RF']
target_column = 'PM2.5'


In [6]:
# Splitting the dataset into features and target variable
X = data[feature_columns].values
y = data[target_column].values


In [7]:
# Normalizing the feature values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# sliding window parameters
window_size = 5  # Number of past time steps considered
output_size = 10  # Number of steps ahead used to predict pollutant concentration


In [9]:
# Create 3D dataset using sliding window technique
def create_dataset(X, y, window_size, output_size):
    X_windowed = []
    y_windowed = []
    for i in range(len(X) - window_size - output_size + 1):
        X_windowed.append(X[i:i+window_size])
        y_windowed.append(y[i+window_size:i+window_size+output_size])
    return np.array(X_windowed), np.array(y_windowed)

X_windowed, y_windowed = create_dataset(X_scaled, y, window_size, output_size)


In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_windowed, test_size=0.2, random_state=42)


In [11]:
# Build the bidirectional LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(units=50, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Bidirectional(LSTM(units=50)))
model.add(Dense(output_size))  # Output layer with 'output_size' neurons for multi-step ahead prediction
model.compile(optimizer='adam', loss='mse')

In [12]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/100

827/827 [==============================] - 2468s 3s/step - loss: 4891.5986 - val_loss: 3870.8667
Epoch 2/100
827/827 [==============================] - 13s 15ms/step - loss: 3282.0125 - val_loss: 2493.7092
Epoch 3/100
827/827 [==============================] - 9s 11ms/step - loss: 2545.7942 - val_loss: 2179.9624
Epoch 4/100
827/827 [==============================] - 7s 8ms/step - loss: 2369.8171 - val_loss: 2110.3740
Epoch 5/100
827/827 [==============================] - 7s 9ms/step - loss: 2217.2007 - val_loss: 1929.9298
Epoch 6/100
827/827 [==============================] - 7s 8ms/step - loss: 2092.2839 - val_loss: 1881.8032
Epoch 7/100
827/827 [==============================] - 7s 8ms/step - loss: 1993.5674 - val_loss: 1727.2281
Epoch 8/100
827/827 [==============================] - 7s 8ms/step - loss: 1904.3510 - val_loss: 1691.4097
Epoch 9/100
827/827 [==============================] - 7s 8ms/step - loss: 1835.9005 - val_loss: 1609.1215
Epoch 10/100
827/827 [=========

In [14]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

207/207 [==============================] - 1s 4ms/step - loss: 1264.8422
Test Loss: 1264.8421630859375
207/207 [==============================] - 1s 4ms/step


In [22]:
from sklearn.metrics import mean_squared_error, r2_score
import scipy.stats as stats

# Calculate Fraction of predictions with a factor of 2 (FAC2)
def fac2(y_true, y_pred):
    within_factor = np.abs(y_true / y_pred) <= 2
    return np.mean(within_factor)

# Calculate Root Mean Square Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Calculate Correlation Coefficient (r)
r = np.corrcoef(np.ravel(y_test), np.ravel(predictions))[0, 1]

# Calculate Mean Gross Error (MGE)
mge = np.mean(np.abs(y_test - predictions))

# Calculate Mean Bias (MB)
mb = np.mean(y_test - predictions)

# Calculate Coefficient of Efficiency (COE)
coe = 1 - (np.sum((y_test - predictions) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2))

# Calculate Coefficient of Determination (R2 score)
r2 = r2_score(y_test, predictions)

# Print the evaluation metrics
print(f'Fraction of predictions with a factor of 2 (FAC2): {fac2(y_test, predictions)}')
print(f'Root Mean Square Error (RMSE): {rmse}')
print(f'Correlation Coefficient (r): {r}')
print(f'Mean Gross Error (MGE): {mge}')
print(f'Mean Bias (MB): {mb}')
print(f'Coefficient of Efficiency (COE): {coe}')
print(f'Coefficient of Determination (R2 score): {r2}')

Fraction of predictions with a factor of 2 (FAC2): 0.9374073513840568
Root Mean Square Error (RMSE): 35.56462288138887
Correlation Coefficient (r): 0.8244117061979126
Mean Gross Error (MGE): 18.787749665850537
Mean Bias (MB): -0.274143013053935
Coefficient of Efficiency (COE): 0.6748135861095224
Coefficient of Determination (R2 score): 0.6742770039185186
